**Breast Cancer Jump Scare: Survival Prediction**

Cadeem's Part: Data Processing

In [1]:
import os
# Find the latest version of spark 3.x from http://www.apache.org/dist/spark/
# and enter as the spark version. For example:
# spark_version = 'spark-3.5.2'
spark_version = 'spark-3.5.2'
os.environ['SPARK_VERSION'] = spark_version

In [2]:
# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

'apt-get' is not recognized as an internal or external command,
operable program or batch file.


The system cannot find the path specified.
'wget' is not recognized as an internal or external command,
operable program or batch file.
tar: Error opening archive: Failed to open '$SPARK_VERSION-bin-hadoop3.tgz'


In [3]:
# Install required Python packages
!pip install scikit-learn #--upgrade


ERROR: Invalid requirement: '#--upgrade'


In [4]:
!pip install scikit-survival  # Install the scikit-survival package

ERROR: Invalid requirement: '#'


In [5]:
# Set environment variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

In [12]:
jup = 234

In [6]:
import findspark

In [11]:
# Initialize Spark
# import findspark
findspark.init()

Exception: Unable to find py4j in /content/spark-3.5.2-bin-hadoop3\python, your SPARK_HOME may not be configured correctly

In [ ]:
import py4j
print(py4j.__version__)

0.10.9.7


In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import OneHotEncoder
from google.colab import drive
from sklearn.impute import SimpleImputer
import sklearn
from sksurv.ensemble import RandomSurvivalForest
import numpy as np
from sklearn.preprocessing import LabelEncoder
import sqlite3

In [ ]:
# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

/usr/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(


In [ ]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles

sp_url = "https://project-04-team-05-uoft-scs-data-visualization.s3.us-east-2.amazonaws.com/METABRIC_RNA_Mutation.csv"

spark.sparkContext.addFile(sp_url)

sp_df = spark.read.option('header', 'true').csv(SparkFiles.get("METABRIC_RNA_Mutation.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")


<ipython-input-11-9d0cb5bee92f>:3: DtypeWarning: Columns (678,688,690,692) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url)


In [ ]:
# Preview the data

sp_df.show(10)

In [ ]:
# Cleaning the DataFrame by selecting the first 31 columns
# Using list slicing to select the first 31 columns by name and filtering on that list
column_names = sp_df.columns[:31]
spdf_clean = df.select(column_names)

# Display the cleaned DataFrame
spdf_clean.show(5)

In [ ]:
# Converting the filtered spark data frame to Pandas
df_clean = spdf_clean.toPandas()

# Preview df_clean
df_clean.head()

,patient_id,age_at_diagnosis,type_of_breast_surgery,cancer_type,cancer_type_detailed,cellularity,chemotherapy,pam50_+_claudin-low_subtype,cohort,er_status_measured_by_ihc,...,nottingham_prognostic_index,oncotree_code,overall_survival_months,overall_survival,pr_status,radio_therapy,3-gene_classifier_subtype,tumor_size,tumor_stage,death_from_cancer
0,0,75.65,MASTECTOMY,Breast Cancer,Breast Invasive Ductal Carcinoma,NaN,0,claudin-low,1.0,Positve,...,6.044,IDC,140.500000,1,Negative,1,ER-/HER2-,22.0,2.0,Living
1,2,43.19,BREAST CONSERVING,Breast Cancer,Breast Invasive Ductal Carcinoma,High,0,LumA,1.0,Positve,...,4.020,IDC,84.633333,1,Positive,1,ER+/HER2- High Prolif,10.0,1.0,Living
2,5,48.87,MASTECTOMY,Breast Cancer,Breast Invasive Ductal Carcinoma,High,1,LumB,1.0,Positve,...,4.030,IDC,163.700000,0,Positive,0,NaN,15.0,2.0,Died of Disease
3,6,47.68,MASTECTOMY,Breast Cancer,Breast Mixed Ductal and Lobular Carcinoma,Moderate,1,LumB,1.0,Positve,...,4.050,MDLC,164.933333,1,Positive,1,NaN,25.0,2.0,Living
4,8,76.97,MASTECTOMY,Breast Cancer,Breast Mixed Ductal and Lobular Carcinoma,High,1,LumB,1.0,Positve,...,6.080,MDLC,41.366667,0,Positive,1,ER+/HER2- High Prolif,40.0,2.0,Died of Disease


In [ ]:
# df_clean.info() ####################### can this me removed?

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1904 entries, 0 to 1903
Data columns (total 31 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   patient_id                      1904 non-null   int64  
 1   age_at_diagnosis                1904 non-null   float64
 2   type_of_breast_surgery          1882 non-null   object 
 3   cancer_type                     1904 non-null   object 
 4   cancer_type_detailed            1889 non-null   object 
 5   cellularity                     1850 non-null   object 
 6   chemotherapy                    1904 non-null   int64  
 7   pam50_+_claudin-low_subtype     1904 non-null   object 
 8   cohort                          1904 non-null   float64
 9   er_status_measured_by_ihc       1874 non-null   object 
 10  er_status                       1904 non-null   object 
 11  neoplasm_histologic_grade       1832 non-null   float64
 12  her2_status_measured_by_snp6    19

In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
# Save the cleaned data
drive.mount('drive')
df_clean.to_csv('Cleaned_Breast_Cancer_data.csv', index=False)
!cp Cleaned_Breast_Cancer_data.csv "drive/My Drive/"

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


Lucas' Part
columns : overall survival vs neoplasm_histologic_grade When a patient's histological grade is on the file, can we predict overall survival based on the model?

neoplasm_histologic_grade = target variable

'chemotherapy' 'hormone_therapy' 'radio_therapy' 'tumor_size', 'tumor_stage', 'age_at_diagnosis', 'type_of_breast_surgery' 'inferred_menopausal_state', 'overall_survival_months', 'overall_survival

'chemotherapy'
'hormone_therapy'
'radio_therapy'
'tumor_size',
'tumor_stage',
'age_at_diagnosis',
'type_of_breast_surgery'
'inferred_menopausal_state',
'overall_survival',

@Lucas, We also need to insert the Logistic regression part here. for the same analysis Overall survival vs

In [ ]:
# Prepare the data by removing NaN
df_clean.dropna(subset = ['type_of_breast_surgery'], inplace=True)
df_clean.shape

(1882, 31)

In [ ]:
# Prepare the OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)

In [ ]:
# Perform one-hot encoding for the 'type_of_breast_surgery' column
encoded_data_lp = encoder.fit_transform(df_clean[['type_of_breast_surgery']])

In [ ]:
# Prepare the dataframe for the encoded data
encoded_df_lp = pd.DataFrame(encoded_data_lp, columns=encoder.get_feature_names_out(['type_of_breast_surgery']))

In [ ]:
# Concatenate the original dataframe with the encoded dataframe
df_lp_1= pd.concat([df_clean, encoded_df_lp], axis=1, join='outer')

In [ ]:
df_lp_1.head()

,patient_id,age_at_diagnosis,type_of_breast_surgery,cancer_type,cancer_type_detailed,cellularity,chemotherapy,pam50_+_claudin-low_subtype,cohort,er_status_measured_by_ihc,...,overall_survival_months,overall_survival,pr_status,radio_therapy,3-gene_classifier_subtype,tumor_size,tumor_stage,death_from_cancer,type_of_breast_surgery_BREAST CONSERVING,type_of_breast_surgery_MASTECTOMY
0,0.0,75.65,MASTECTOMY,Breast Cancer,Breast Invasive Ductal Carcinoma,NaN,0.0,claudin-low,1.0,Positve,...,140.500000,1.0,Negative,1.0,ER-/HER2-,22.0,2.0,Living,0.0,1.0
1,2.0,43.19,BREAST CONSERVING,Breast Cancer,Breast Invasive Ductal Carcinoma,High,0.0,LumA,1.0,Positve,...,84.633333,1.0,Positive,1.0,ER+/HER2- High Prolif,10.0,1.0,Living,1.0,0.0
2,5.0,48.87,MASTECTOMY,Breast Cancer,Breast Invasive Ductal Carcinoma,High,1.0,LumB,1.0,Positve,...,163.700000,0.0,Positive,0.0,NaN,15.0,2.0,Died of Disease,0.0,1.0
3,6.0,47.68,MASTECTOMY,Breast Cancer,Breast Mixed Ductal and Lobular Carcinoma,Moderate,1.0,LumB,1.0,Positve,...,164.933333,1.0,Positive,1.0,NaN,25.0,2.0,Living,0.0,1.0
4,8.0,76.97,MASTECTOMY,Breast Cancer,Breast Mixed Ductal and Lobular Carcinoma,High,1.0,LumB,1.0,Positve,...,41.366667,0.0,Positive,1.0,ER+/HER2- High Prolif,40.0,2.0,Died of Disease,0.0,1.0


In [ ]:
'inferred_menopausal_state'

'inferred_menopausal_state'

In [ ]:
# Prepare the data by removing NaN
df_lp_1.dropna(subset = ['inferred_menopausal_state'], inplace=True)
df_lp_1.shape

(1882, 33)

In [ ]:
df_lp_1['inferred_menopausal_state'].isnull().values.any()

False

In [ ]:
df_lp_1.dropna(inplace=True)

In [ ]:
df_lp_1.isnull().values.any()

False

In [ ]:
# Perform one-hot encoding for the 'type_of_breast_surgery' column
encoded_data_lp_1 = encoder.fit_transform(df_lp_1[['inferred_menopausal_state']])

In [ ]:
# Prepare the dataframe for the encoded data
encoded_df_lp2 = pd.DataFrame(encoded_data_lp_1, columns=encoder.get_feature_names_out(['inferred_menopausal_state']))

In [ ]:
encoded_df_lp2.isnull().values.any()

False

In [ ]:
df_lp_1 = df_lp_1.reset_index(drop=True)
encoded_df_lp2 = encoded_df_lp2.reset_index(drop=True)

In [ ]:
`# Concatenate the original dataframe with the encoded dataframe
df_lp= pd.concat([df_lp_1, encoded_df_lp2], axis=1, join='outer')

In [ ]:
df_lp.head()

,patient_id,age_at_diagnosis,type_of_breast_surgery,cancer_type,cancer_type_detailed,cellularity,chemotherapy,pam50_+_claudin-low_subtype,cohort,er_status_measured_by_ihc,...,pr_status,radio_therapy,3-gene_classifier_subtype,tumor_size,tumor_stage,death_from_cancer,type_of_breast_surgery_BREAST CONSERVING,type_of_breast_surgery_MASTECTOMY,inferred_menopausal_state_Post,inferred_menopausal_state_Pre
0,2.0,43.19,BREAST CONSERVING,Breast Cancer,Breast Invasive Ductal Carcinoma,High,0.0,LumA,1.0,Positve,...,Positive,1.0,ER+/HER2- High Prolif,10.0,1.0,Living,1.0,0.0,0.0,1.0
1,8.0,76.97,MASTECTOMY,Breast Cancer,Breast Mixed Ductal and Lobular Carcinoma,High,1.0,LumB,1.0,Positve,...,Positive,1.0,ER+/HER2- High Prolif,40.0,2.0,Died of Disease,0.0,1.0,1.0,0.0
2,10.0,78.77,MASTECTOMY,Breast Cancer,Breast Invasive Ductal Carcinoma,Moderate,0.0,LumB,1.0,Positve,...,Positive,1.0,ER+/HER2- High Prolif,31.0,4.0,Died of Disease,0.0,1.0,1.0,0.0
3,28.0,86.41,BREAST CONSERVING,Breast Cancer,Breast Invasive Ductal Carcinoma,Moderate,0.0,LumB,1.0,Positve,...,Negative,1.0,ER+/HER2- High Prolif,16.0,2.0,Died of Other Causes,1.0,0.0,1.0,0.0
4,35.0,84.22,MASTECTOMY,Breast Cancer,Breast Invasive Lobular Carcinoma,High,0.0,Her2,1.0,Negative,...,Negative,0.0,ER+/HER2- High Prolif,28.0,2.0,Died of Disease,0.0,1.0,1.0,0.0


In [ ]:
df_lp.isnull().values.any()

False

In [ ]:
df_lp.columns

Index(['patient_id', 'age_at_diagnosis', 'type_of_breast_surgery',
       'cancer_type', 'cancer_type_detailed', 'cellularity', 'chemotherapy',
       'pam50_+_claudin-low_subtype', 'cohort', 'er_status_measured_by_ihc',
       'er_status', 'neoplasm_histologic_grade',
       'her2_status_measured_by_snp6', 'her2_status',
       'tumor_other_histologic_subtype', 'hormone_therapy',
       'inferred_menopausal_state', 'integrative_cluster',
       'primary_tumor_laterality', 'lymph_nodes_examined_positive',
       'mutation_count', 'nottingham_prognostic_index', 'oncotree_code',
       'overall_survival_months', 'overall_survival', 'pr_status',
       'radio_therapy', '3-gene_classifier_subtype', 'tumor_size',
       'tumor_stage', 'death_from_cancer',
       'type_of_breast_surgery_BREAST CONSERVING',
       'type_of_breast_surgery_MASTECTOMY', 'inferred_menopausal_state_Post',
       'inferred_menopausal_state_Pre'],
      dtype='object')

In [ ]:
df_lp.drop(columns=['inferred_menopausal_state'], inplace=True)

In [ ]:
# Prepare the data to remove NaN for the Random Survival Forest model
df_lp.dropna(subset = ['overall_survival'], inplace=True)
df_lp.dropna(subset = ['overall_survival_months'], inplace=True)

In [ ]:
df_lp['overall_survival'].isnull().values.any()

False

In [ ]:
df_lp.isnull().values.any()

False

In [ ]:
# Prepare logistic regression model
X = df_lp[['chemotherapy', 'hormone_therapy', 'radio_therapy', 'tumor_size', 'tumor_stage', 'age_at_diagnosis', 'type_of_breast_surgery_BREAST CONSERVING', 'type_of_breast_surgery_MASTECTOMY', 'inferred_menopausal_state_Post',
       'inferred_menopausal_state_Pre']]
y = df_lp['overall_survival']

In [ ]:
X.isnull().values.any()

False

In [ ]:
# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# create a logistic regression model
log_reg_lp = LogisticRegression()

In [ ]:
# train the model on the training data
log_reg_lp.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
# make predictions on the testing data
y_pred = log_reg_lp.predict(X_test)

In [ ]:
# evaluate the model using accuracy score
accuracy = log_reg_lp.score(X_test, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.6164383561643836


In [ ]:
c_matrix = confusion_matrix(y_test, y_pred)
c_matrix_df = pd.DataFrame(
    c_matrix, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

In [ ]:
# Print the classification report for the model
print("Classification Report")
print(classification_report(y_test, y_pred))

Classification Report
              precision    recall  f1-score   support

         0.0       0.67      0.65      0.66       124
         1.0       0.56      0.58      0.57        95

    accuracy                           0.62       219
   macro avg       0.61      0.61      0.61       219
weighted avg       0.62      0.62      0.62       219



In [ ]:
# Prepare the data for the Random Survival Forest model
X_rsf = df_lp[['chemotherapy', 'hormone_therapy', 'radio_therapy', 'tumor_size', 'tumor_stage', 'age_at_diagnosis', 'type_of_breast_surgery_BREAST CONSERVING', 'type_of_breast_surgery_MASTECTOMY']]
y_rsf = np.array(list(zip(df_lp['overall_survival'], df_lp['overall_survival_months'])),
                 dtype=[('overall_survival', '?'), ('overall_survival_months', '<f8')])

In [ ]:
# Perform train-test split for the Random Survival Forest model
from sklearn.model_selection import train_test_split
X_train_rsf, X_test_rsf, y_train_rsf, y_test_rsf = train_test_split(X_rsf, y_rsf, test_size=0.2, random_state=42)

In [ ]:
# Create a Random Survival Forest model
rsf = RandomSurvivalForest(n_estimators=100, random_state=42)

In [ ]:
# Initialize LabelEncoder
le = LabelEncoder()

# Iterate over columns in X_train and encode any string columns
for col in X_train_rsf.columns:
    if X_train_rsf[col].dtype == 'object':
        X_train_rsf[col] = le.fit_transform(X_train_rsf[col])

# Fit the model to the training data with encoded features
rsf.fit(X_train_rsf, y_train_rsf)

RandomSurvivalForest(random_state=42)

In [ ]:
# Predict the survival probability at a given time point
time_point = 730  # 2 year
survival_prob = rsf.predict_survival_function(X_train_rsf, time_point)

In [ ]:
survival_prob_df = pd.DataFrame(survival_prob)
survival_prob_df.head()

,0,1,2,3,4,5,6,7,8,9,...,815,816,817,818,819,820,821,822,823,824
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.043182,0.043182,0.043182,0.043182,0.043182,0.043182,0.043182,0.043182,0.043182,0.043182
1,0.999246,0.999246,0.999246,0.999246,0.999246,0.999246,0.999246,0.999246,0.999246,0.999246,...,0.048958,0.048958,0.048958,0.048958,0.048958,0.048958,0.048958,0.048958,0.048958,0.048958
2,1.000000,1.000000,1.000000,0.999470,0.999470,0.999470,0.999470,0.999470,0.999470,0.999470,...,0.076084,0.076084,0.076084,0.076084,0.027084,0.027084,0.027084,0.027084,0.027084,0.026064
3,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.368159,0.368159,0.366359,0.366359,0.365993,0.365993,0.365993,0.365993,0.365993,0.365993
4,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.057000,0.057000,0.057000,0.057000,0.057000,0.057000,0.057000,0.052000,0.042000,0.042000


In [ ]:
survival_prob_df.shape

(873, 825)

In [ ]:
df_lp_list = []
for i, patient_probs in enumerate(survival_prob):
    prob_at_time_point = patient_probs[time_point]  # Get probability at the specified time point
    df_lp_list.append(prob_at_time_point)

In [ ]:
# Print the survival probability for each patient in the test set
for i, patient_probs in enumerate(survival_prob):
    prob_at_time_point = patient_probs[time_point]  # Get probability at the specified time point
    print(f"Patient {i}: Survival probability at {time_point} days = {prob_at_time_point:.4f}")

Patient 0: Survival probability at 730 days = 0.0858
Patient 1: Survival probability at 730 days = 0.1285
Patient 2: Survival probability at 730 days = 0.3253
Patient 3: Survival probability at 730 days = 0.4830
Patient 4: Survival probability at 730 days = 0.1415
Patient 5: Survival probability at 730 days = 0.5403
Patient 6: Survival probability at 730 days = 0.2274
Patient 7: Survival probability at 730 days = 0.1089
Patient 8: Survival probability at 730 days = 0.2130
Patient 9: Survival probability at 730 days = 0.7815
Patient 10: Survival probability at 730 days = 0.7516
Patient 11: Survival probability at 730 days = 0.6857
Patient 12: Survival probability at 730 days = 0.2344
Patient 13: Survival probability at 730 days = 0.3073
Patient 14: Survival probability at 730 days = 0.6942
Patient 15: Survival probability at 730 days = 0.3361
Patient 16: Survival probability at 730 days = 0.9324
Patient 17: Survival probability at 730 days = 0.5634
Patient 18: Survival probability at 73

In [ ]:
# Creating a dataframe of patients survival probability

patients_info = []

for i, patient_probs in enumerate(survival_prob):
  patient = {}
  patient['patient_id'] = i
  patient['survival probability at 180 days'] = survival_prob[i][180]
  patient['survival probability at 365 days'] = survival_prob[i][365]
  patient['survival probability at 545 days'] = survival_prob[i][545]
  patient['survival probability at 730 days'] = survival_prob[i][730]
  patients_info.append(patient)

  patients_survive_info_df = pd.DataFrame(patients_info)





In [ ]:
patients_survive_info_df.head()

,patient_id,survival probability at 180 days,survival probability at 365 days,survival probability at 545 days,survival probability at 730 days
0,0,0.998571,0.879471,0.216091,0.085850
1,1,0.987496,0.921208,0.811910,0.128509
2,2,0.999470,0.987444,0.764962,0.325275
3,3,1.000000,0.983698,0.939805,0.483045
4,4,0.967509,0.829850,0.495934,0.141528


In [ ]:
# Save the cleaned data

patients_survive_info_df.to_csv('Patients_survival.csv', index=False)
!cp Patients_survival.csv "drive/My Drive/"

Please double check/review/validate if the below is suitable @Jaz, @Dem and @Adi and let me know

In [ ]:
# Create a connection to the database
conn = sqlite3.connect('survival_probabilities.db')

In [ ]:
# Create a cursor object
c = conn.cursor()

In [ ]:
# Create a table to store the survival probabilities
c.execute('''
    CREATE TABLE IF NOT EXISTS survival_probabilities
    (patient_id INTEGER, time_point INTEGER, survival_probability REAL)
''')

In [ ]:
# Define the time points
time_points = [365, 730, 1095]  # 1 year, 2 years, 3 years

In [ ]:
# Predict the survival probability at each time point
for time_point in time_points:
    survival_prob = rsf.predict_survival_function(X_train_rsf, time_point)
    for i, patient_probs in enumerate(survival_prob):
        # Iterate over the time points within each patient's survival probabilities
        for t in range(len(patient_probs)):
            prob_at_time_point = patient_probs[t]  # Get probability at the current time point
            # Insert the survival probability into the database
            c.execute('INSERT INTO survival_probabilities VALUES (?, ?, ?)', (i, t, prob_at_time_point))
# Commit the changes
conn.commit()

# Close the connection
conn.close()

Make sure to run all cells in order everytime you call them

In [ ]:
# Create a connection to the database
conn = sqlite3.connect('survival_probabilities.db')

# Create a cursor object
c = conn.cursor()

In [ ]:
# Query the database
c.execute('SELECT survival_probability FROM survival_probabilities WHERE patient_id = ? AND time_point = ?', (0, 730))

In [ ]:
# Execute a SELECT query to get the data
cursor.execute('SELECT * FROM survival_probabilities.db')

# Fetch all rows from the table
rows = cursor.fetchall()

# Open a CSV file in write mode
with open('output_file.csv', 'w', newline='') as file:
    writer = csv.writer(file)

    # Write the column headers if needed
    column_names = [description[0] for description in cursor.description]
    writer.writerow(column_names)

    # Write all rows to the CSV file
    writer.writerows(rows)

NameError: name 'cursor' is not defined

In [ ]:
# Fetch the result
result = c.fetchone()

# Print the result
print(result[0])

# Close the connection
conn.close()

0.08584965202398998


Aditya's Part

 outcome of breast cancer treatment type vs cancer type

What type of cancer treatment can we predict from our model based on the type of cancer diagnosis given to a patient?

In [ ]:
column_arr = df_clean.columns
print(column_arr)


In [ ]:
# Define a function to encode the treatment types
def encode_treatment(row):
    if row['chemotherapy'] == 0 and row['radio_therapy'] == 0 and row['hormone_therapy'] == 0:
        return 0  # No therapy
    elif row['chemotherapy'] == 1 and row['radio_therapy'] == 0 and row['hormone_therapy'] == 0:
        return 1  # Chemotherapy only
    elif row['chemotherapy'] == 0 and row['radio_therapy'] == 1 and row['hormone_therapy'] == 0:
        return 2  # Radiotherapy only
    elif row['chemotherapy'] == 0 and row['radio_therapy'] == 0 and row['hormone_therapy'] == 1:
        return 3  # Hormone therapy only
    elif row['chemotherapy'] == 1 and row['radio_therapy'] == 1 and row['hormone_therapy'] == 0:
        return 4  # Chemotherapy and Radiotherapy only
    elif row['chemotherapy'] == 0 and row['radio_therapy'] == 1 and row['hormone_therapy'] == 1:
        return 5  # Radiotherapy and Hormone therapy only
    elif row['chemotherapy'] == 1 and row['radio_therapy'] == 0 and row['hormone_therapy'] == 1:
        return 6  # Chemotherapy and Hormone therapy only
    elif row['chemotherapy'] == 1 and row['radio_therapy'] == 1 and row['hormone_therapy'] == 1:
        return 7  # All three therapies


In [ ]:
# Apply the function to create the new column
df_clean['treatment_type'] = df_clean.apply(encode_treatment, axis=1)
df_clean.head()

In [ ]:
df_clean_modelling = df_clean.drop(columns=['chemotherapy', 'radio_therapy', 'hormone_therapy', 'patient_id', 'death_from_cancer','tumor_stage','3-gene_classifier_subtype'])

In [ ]:
df_clean_modelling.shape

In [ ]:
na_count_df_clean_modelling = df_clean_modelling.isna().sum()
print(na_count_df_clean_modelling)

In [ ]:
df_clean_modelling = df_clean_modelling.dropna()

In [ ]:
na_count_df_clean_modelling = df_clean_modelling.isna().sum()
print(na_count_df_clean_modelling)

In [ ]:
df_clean_modelling.columns

In [ ]:
# Now drop the individual therapy columns since we have collated them into one
X = df_clean_modelling.drop(columns=['treatment_type'])
y = df_clean_modelling['treatment_type']


In [ ]:
# Categorical columns to be encoded
categorical_cols = X.select_dtypes(include=['object', 'category']).columns

# Numerical columns to be scaled
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns

In [ ]:
from sklearn.compose import ColumnTransformer

In [ ]:
# Preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)])  # Handle unknown categories

# Fit and transform the entire dataset
X_transformed = preprocessor.fit_transform(X)


In [ ]:
# Splitting the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Categorical columns to be encoded
categorical_cols = X.select_dtypes(include=['object', 'category']).columns

# Numerical columns to be scaled
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns


In [ ]:
# Fit the preprocessor on training data
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models


In [ ]:
# Build the model
model1 = models.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(8, activation='softmax')  # Output layer has 8 neurons, one for each treatment type
])
# Check the structure of the model
model1.summary()

In [ ]:
# Compile the model
model1.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Train the model
model1.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


In [ ]:
# Build a more complex model
model2 = models.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dropout(0.5),  # Regularization with Dropout
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(32, activation='relu'),
    layers.Dense(8, activation='softmax')  # Output layer for 8 classes
])
# Check the structure of the model
model2.summary()

In [ ]:
# Compile the model
model2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model2.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))


In [ ]:
# Compile the model
model2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model2.fit(X_train, y_train, epochs=75, batch_size=64, validation_data=(X_test, y_test))


In [ ]:
# Compile the model
model2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model2.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_test, y_test))

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Initialize the logistic regression model
logreg = LogisticRegression(max_iter=1000, multi_class='multinomial', solver='lbfgs')


In [ ]:
# Train the logistic regression model
logreg.fit(X_train, y_train)

# Make predictions
y_pred = logreg.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Logistic Regression Accuracy: {accuracy:.2f}")


Testing the Model Trained above:
a dummy dataset with 10 patients is created. This dataset will include all the features used to train the model. After creating the dataset, a loop is created to run through it and make predictions using one of the trained models. (Model 2)
Dummy Dataset:
The dummy_data DataFrame simulates 10 patients with randomly generated features that match those used in the model training.
Preprocessing:
The dummy dataset is preprocessed using the same preprocessor pipeline that was used for the training data. This ensures consistency in the input data format.
Predictions:
The model1.predict() function returns the predicted probabilities for each class. np.argmax(predictions, axis=1) is used to determine the class with the highest probability, which corresponds to the predicted treatment type.
The loop iterates through each patient, printing the predicted treatment type using the treatment_mapping dictionary to convert numerical predictions back to human-readable labels.


In [ ]:
col_df = list(df_clean_modelling.columns)
print(col_df)

In [ ]:
num_columns = len(df_clean_modelling.columns)
print(num_columns)

In [ ]:
unique_values = {col: df_clean_modelling[col].unique() for col in df_clean_modelling.columns}
print(unique_values)

In [ ]:
import numpy as np
import random

# Random seed for reproducibility
np.random.seed(42)

# Number of dummy patients
num_patients = 100

# Initialize an empty list to store the patient data
dummy_data = []

# Generate the dummy dataset
for _ in range(num_patients):
  patient_data = {col: random.choice(unique_values[col]) for col in unique_values.keys()}
  dummy_data.append(patient_data)

dummy_data_df = pd.DataFrame(dummy_data)

# Display the dummy dataset
dummy_data_df.head()



In [ ]:
# Preprocess the dummy dataset using the same preprocessor
dummy_X = preprocessor.transform(dummy_data_df)


In [ ]:
# Assuming model1 is the trained simple neural network
dummy_predictions_model1 = model1.predict(dummy_X)
dummy_predictions_model1 = np.argmax(dummy_predictions_model1, axis=1)  # Convert to class labels

# Assuming model2 is the trained complex neural network
dummy_predictions_model2 = model2.predict(dummy_X)
dummy_predictions_model2 = np.argmax(dummy_predictions_model2, axis=1)  # Convert to class labels

# Assuming logreg is the trained logistic regression model
dummy_predictions_logreg = logreg.predict(dummy_X)


In [ ]:
# Map the predictions back to the treatment types
treatment_mapping = {
    0: "No therapy",
    1: "Chemotherapy only",
    2: "Radiotherapy only",
    3: "Hormone therapy only",
    4: "Chemotherapy and Radiotherapy only",
    5: "Radiotherapy and Hormone therapy only",
    6: "Chemotherapy and Hormone therapy only",
    7: "Chemotherapy, Radiotherapy, and Hormone therapy"
}


In [ ]:
data = []

In [ ]:
# Iterate through the predictions and store the information in the list
for i, prediction in enumerate(dummy_predictions_model2):
    actual_treatment = treatment_mapping[dummy_data_df['treatment_type'][i]]
    predicted_treatment_model1 = treatment_mapping[dummy_predictions_model1[i]]
    predicted_treatment_model2 = treatment_mapping[prediction]
    predicted_treatment_logistic = treatment_mapping[dummy_predictions_logreg[i]]
    data.append({
        'Patient Number': i + 1,
        'Actual Treatment': actual_treatment,
        'Predicted Treatment Model1': predicted_treatment_model1,
        'Predicted Treatment Model2': predicted_treatment_model2,
        'Predicted Treatment Logistic Regression': predicted_treatment_logistic
    })


In [ ]:
# Create a DataFrame from the list
df_dummy_pred_act = pd.DataFrame(data)
df_dummy_pred_act.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for col in ['Actual Treatment', 'Predicted Treatment Model1', 'Predicted Treatment Model2', 'Predicted Treatment Logistic Regression']:
    df_dummy_pred_act[col] = le.fit_transform(df_dummy_pred_act[col])

In [ ]:
df_dummy_pred_act.head()

In [ ]:
#Importing plotly
from matplotlib import pyplot as plt
from matplotlib.pyplot import subplots

In [ ]:
# Plotting
fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(18, 12), sharex=True, sharey=True)

# Plot for Model1
axes[0].plot(df_dummy_pred_act['Patient Number'], df_dummy_pred_act['Actual Treatment'], label='Actual', color='blue')
axes[0].plot(df_dummy_pred_act['Patient Number'], df_dummy_pred_act['Predicted Treatment Model1'], label='Model1', color='red')
axes[0].set_title('Actual vs Model1')
axes[0].set_xlabel('Patient Index')
# axes[0].set_ylabel('Treatment_type', rotation=0, ha='right')
axes[0].set_yticks(range(len(treatment_mapping)))
axes[0].set_yticklabels(list(treatment_mapping.values()))
axes[0].grid(True)
axes[0].legend()

# Plot for Model2
axes[1].plot(df_dummy_pred_act['Patient Number'], df_dummy_pred_act['Actual Treatment'], label='Actual', color='blue')
axes[1].plot(df_dummy_pred_act['Patient Number'], df_dummy_pred_act['Predicted Treatment Model2'], label='Model2', color='green')
axes[1].set_title('Actual vs Model2')
axes[1].set_xlabel('Patient Index')
# axes[1].set_ylabel('Treatment_type', rotation=0, ha='right')
axes[1].set_yticks(range(len(treatment_mapping)))
axes[1].set_yticklabels(list(treatment_mapping.values()))
axes[1].grid(True)
axes[1].legend()

# Plot for Logistic Regression
axes[2].plot(df_dummy_pred_act['Patient Number'], df_dummy_pred_act['Actual Treatment'], label='Actual', color='blue')
axes[2].plot(df_dummy_pred_act['Patient Number'], df_dummy_pred_act['Predicted Treatment Logistic Regression'], label='Logistic Regression', color='purple')
axes[2].set_title('Actual vs Logistic Regression')
axes[2].set_xlabel('Patient Index')
# axes[2].set_ylabel('Treatment_type', rotation=0, ha='right')
axes[2].set_yticks(range(len(treatment_mapping)))
axes[2].set_yticklabels(list(treatment_mapping.values()))
axes[2].grid(True)
axes[2].legend()

plt.show()


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

# Calculate metrics for each model
accuracy_model1 = accuracy_score(df_dummy_pred_act['Actual Treatment'], df_dummy_pred_act['Predicted Treatment Model1'])
precision_model1 = precision_score(df_dummy_pred_act['Actual Treatment'], df_dummy_pred_act['Predicted Treatment Model1'], average='weighted')
recall_model1 = recall_score(df_dummy_pred_act['Actual Treatment'], df_dummy_pred_act['Predicted Treatment Model1'], average='weighted')

accuracy_model2 = accuracy_score(df_dummy_pred_act['Actual Treatment'], df_dummy_pred_act['Predicted Treatment Model2'])
precision_model2 = precision_score(df_dummy_pred_act['Actual Treatment'], df_dummy_pred_act['Predicted Treatment Model2'], average='weighted')
recall_model2 = recall_score(df_dummy_pred_act['Actual Treatment'], df_dummy_pred_act['Predicted Treatment Model2'], average='weighted')

accuracy_logistic = accuracy_score(df_dummy_pred_act['Actual Treatment'], df_dummy_pred_act['Predicted Treatment Logistic Regression'])
precision_logistic = precision_score(df_dummy_pred_act['Actual Treatment'], df_dummy_pred_act['Predicted Treatment Logistic Regression'], average='weighted')
recall_logistic = recall_score(df_dummy_pred_act['Actual Treatment'], df_dummy_pred_act['Predicted Treatment Logistic Regression'], average='weighted')

# Print metrics
print(f"Model 1 - Accuracy: {accuracy_model1}, Precision: {precision_model1}, Recall: {recall_model1}")
print(f"Model 2 - Accuracy: {accuracy_model2}, Precision: {precision_model2}, Recall: {recall_model2}")
print(f"Logistic Regression - Accuracy: {accuracy_logistic}, Precision: {precision_logistic}, Recall: {recall_logistic}")


In [ ]:
# Classification Report
print("Classification Report for Model 1:")
print(classification_report(actual_treatment, dummy_predictions_model1))
print("Classification Report for Model 2:")
print(classification_report(actual_treatment, dummy_predictions_model2))
print("Classification Report for Logistic Regression:")
print(classification_report(actual_treatment, dummy_predictions_logreg))

***Jas' Part ***